In [1]:
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

import torch
import torchvision.transforms as transforms
import torch.nn as nn
from torch.utils.data import DataLoader

import numpy as np
import sklearn.metrics
import pandas as pd

import dataset
import data_reader
import plots

In [2]:
# Choosing device for tensor processing

if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("Using GPU :", device)
else:
    device = torch.device("cpu")
    print("Using CPU ")

Using GPU : cuda:0


In [3]:
EXP_NAME = "test_normalization"#"256_tcia_25ep"#"25ep"#"tcia_25ep"#

DATA_SET_NAME = f"data_set_X20_100%_SPLIT"#"tcia_data_set_SPLIT"#"train_256_tcia_data_set_SPLIT"#"mhmc"#"test_tcia_data_set_SPLIT"#

In [4]:
def wsi_class(wsi_data_set):
    global net
    """
    Input: 
    - wsi_data_set, patches from a given WSI, dataset object
    - y, patch labels
    Outputs:
    - output, Prediction
    """
    mean_gtex = [144.84250856,  90.71206166, 128.46787316]
    std_gtex = [61.96567854, 60.13392162, 51.13692362]
    normalize_gtex = transforms.Normalize(mean=mean_gtex, std=std_gtex)

    mean_tcga = [190.30330768, 142.36479088, 173.17902561]
    std_tcga = [38.98271104, 45.96033188, 35.20046394] 
    normalize_tcga = transforms.Normalize(mean=mean_tcga, std=std_tcga)

    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]
    normalize = transforms.Normalize(mean=mean, std=std)

    wsi_dataloader = DataLoader(wsi_data_set, batch_size=64)
    outputs = []
    loss = 0

    for batch_X, batch_y, case_ids in iter(wsi_dataloader):
        batch_X, batch_y = batch_X.type(torch.FloatTensor).to(device).permute(0, 3, 2, 1), batch_y.type(torch.FloatTensor).to(device) 

        # NORMALIZATION
        for i, (case_id, x) in enumerate(zip(case_ids, batch_X)):
            if "TCGA" in case_id:
                batch_X[i] = normalize_tcga(batch_X[i])
                batch_X[i] = normalize(batch_X[i])
            elif "GTEX" in case_id:
                batch_X[i] = normalize_gtex(batch_X[i])
                batch_X[i] = normalize(batch_X[i])
            else:
                batch_X[i] = normalize(batch_X[i]/255)
    
        output = net(batch_X)
        outputs.extend(output.cpu())
        
        loss += nn.CrossEntropyLoss()(output, batch_y).cpu()

    #print(outputs)

    y_pred = [torch.argmax(i) for i in outputs] # 1 means positive diagnosis: (1,0) => 1
    prob_neg = y_pred.count(0)/len(y_pred)
    prob_pos = y_pred.count(1)/len(y_pred)

    output = [prob_neg, prob_pos]
    loss = loss/len(wsi_data_set)

    return output, loss

In [5]:
n_splits = 3
tot_conf = np.zeros((2,2))
tot_out, tot_labels, accs, f1s, losses, baccs = [], [], [], [], [], []

for SPLIT in range(n_splits):

    test_out = pd.DataFrame(columns=["Sample ID", "Output", "Label"])
    
    if SPLIT == n_splits-1: # For the final split for validation we take the firs one for test
        SPLIT_NAME = DATA_SET_NAME + f"{0}" # Test set 
    else:
        SPLIT_NAME = DATA_SET_NAME + f"{SPLIT+1}" # Test set
    
    MODEL_NAME = EXP_NAME + f"{SPLIT}"
    
    MODEL_NAME = "test_normalization1"#"train_512_gtex+gdc1"

    print(SPLIT_NAME)
    net = torch.load(f"C:\\Users\\Alejandro\\Desktop\\heterogeneous-data\\results\\WSI\\models\\{MODEL_NAME}.pth") # Model loading

    if "mhmc" in DATA_SET_NAME:    
        split = open(f"C:\\Users\\Alejandro\\Desktop\\heterogeneous-data\\splits\\mhmc_testsplit{SPLIT}.txt", "r").read().split()
        path = "D:/data/WSI/MHMC/TMA"
        X, y, case_ids = data_reader.read_mhmc(path, split)
        
        sample_ids = ["_".join(case_id.split("_")[:-2]) for case_id in case_ids] # Taking only the sample_id, not patch id

    else:
        X, y, _ , case_ids = data_reader.read_lmdb(f"D:\data\WSI\patches\{SPLIT_NAME}")

        sample_ids = [case_id.split("_")[1] for case_id in case_ids] # Taking only the sample_id, not patch id

    outputs, loss, labels = [], [], []
    unique_sample_ids = np.unique(sample_ids)
    sample_ids = np.array(sample_ids)

    for unique_sample_id in tqdm(unique_sample_ids):

        net.eval()
        with torch.no_grad():
    
            ii = np.where(sample_ids == unique_sample_id)[0]

            wsi_data_set = dataset.PatchDataset([], [], [])

            wsi_data_set.inputs.extend(X[ii[0]:ii[-1]])# Taking the patches from a given wsi
            wsi_data_set.labels.extend(y[ii[0]:ii[-1]])
            wsi_data_set.case_ids.extend(sample_ids)

            if len(wsi_data_set) == 0:
                print("No patches")
                outputs.append([np.nan,np.nan])
                labels.append(np.argmax(y[ii[0]]))
                pass
            
            else:
                output = wsi_class(wsi_data_set)
                outputs.append(output[0])
                loss.append(output[1])
                labels.append(np.argmax(y[ii[0]])) # OHE to binary

            del wsi_data_set

    loss = np.mean(loss)

    out = [np.argmax(output) for output in outputs]

    conf_m = sklearn.metrics.confusion_matrix(labels, out, labels=[0, 1])
    acc = sklearn.metrics.accuracy_score(labels, out)
    bacc = sklearn.metrics.balanced_accuracy_score(labels, out)
    f1 = sklearn.metrics.f1_score(labels, out,average="macro")
    auc = 0 #sklearn.metrics.roc_auc_score(y_true, y_pred, average= "micro", multi_class="ovr") #!!!!

    tot_out += out
    tot_labels += labels

    print("Mean Loss: ", loss)
    print("Acc: ", acc)
    print("Bacc: ", bacc)
    print("f1: ", f1)
    print("CONF: \n", conf_m, "\n")

    tot_conf += conf_m

    plots.plot_conf(SPLIT_NAME, conf_m)
    
    '''
    test_out["Sample ID"] = unique_sample_ids # The samples are ordered since the loop goes through unique_sample_ids
    test_out["Output"] = [output[0] for output in outputs]
    test_out["Label"] = labels

    #test_out.to_csv(f"D:\data\Outputs\WSI\PDAC_test_{SPLIT+1}.csv")'''

    accs.append(acc)
    baccs.append(bacc)
    f1s.append(f1)
    losses.append(loss)

tot_acc = np.mean(accs)
tot_bacc = np.mean(baccs)
tot_f1 = np.mean(f1s)
tot_loss = np.mean(losses)

err_acc = np.std(accs)
err_bacc = np.std(baccs)
err_f1 = np.std(f1s)
err_loss = np.std(losses)

print(tot_conf)

data_set_X20_100%_SPLIT1
Read lmdb
8207


  0%|          | 0/52 [00:00<?, ?it/s]

c:\Users\Alejandro\anaconda3\envs\openslide\lib\site-packages\torch\utils\data\_utils\collate.py:146: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_numpy.cpp:178.)
  return default_collate([torch.as_tensor(b) for b in batch])


Mean Loss:  0.012045246
Acc:  0.6538461538461539
Bacc:  0.5
f1:  0.39534883720930236
CONF: 
 [[34  0]
 [18  0]] 

data_set_X20_100%_SPLIT2
Read lmdb


In [ ]:
print("ACC:", round(tot_acc,3)," +/- ", round(err_acc,3))
print("BACC:", round(tot_bacc,3)," +/- ", round(err_bacc,3))
print("F1:", round(tot_f1,3)," +/- ", round(err_f1,3))

plots.plot_conf("10CV", tot_conf)

In [ ]:
roc = []

# ROC
for i in reversed(np.linspace(0,1,10000)):
    out = [1 if output>i else 0 for output in outputs]
    conf_m = confusion_matrix(labels, out, labels=[0, 1])
    roc.append(conf_m[1][1]/(conf_m[1][0]+conf_m[1][1]))

In [ ]:
plots.plot_roc(SPLIT_NAME, roc)

In [ ]:
n_splits = 10
tot_conf = np.zeros((2,2))
tot_out, tot_labels = [], []

#Generating the training outputs PDAC_TEST_(MODEL NUMBER)_(SPLIT OUTPUT)

for TEST_SPLIT in [0,1,2,3]:#range(n_splits):

    MODEL_NAME = EXP_NAME + f"{TEST_SPLIT}"
    net = torch.load(f"C:\\Users\\Alejandro\\Desktop\\heterogeneous-data\\results\\WSI\\models\\{MODEL_NAME}.pth") # Model loading

    for SPLIT in range(n_splits):
        #This has already been executed
        if SPLIT+1==TEST_SPLIT:
            pass

        test_out = pd.DataFrame(columns=["Sample ID", "Output", "Label"])
        
        if SPLIT == n_splits-1: # For the final split for validation we take the first one for test
            SPLIT_NAME = DATA_SET_NAME + f"{0}" # Test set 
        else:
            SPLIT_NAME = DATA_SET_NAME + f"{SPLIT+1}" # Test set

        print(SPLIT_NAME)

        X, y, _ , case_ids = data_reader.read_lmdb(f"D:\data\WSI\patches\{SPLIT_NAME}")

        sample_ids = [case_id.split("_")[1] for case_id in case_ids] # Taking only the sample_id, not patch id

        outputs, loss, labels = [], [], []
        unique_sample_ids = np.unique(sample_ids)
        sample_ids = np.array(sample_ids)

        for unique_sample_id in tqdm(unique_sample_ids):

            net.eval()
            with torch.no_grad():
        
                ii = np.where(sample_ids == unique_sample_id)[0]

                wsi_data_set = dataset.PatchDataset([], [], [])

                wsi_data_set.inputs.extend(X[ii[0]:ii[-1]])# Taking the patches from a given wsi
                wsi_data_set.labels.extend(y[ii[0]:ii[-1]])
                wsi_data_set.case_ids.extend(sample_ids)

                if len(wsi_data_set) == 0:
                    print("No patches")
                    outputs.append([np.nan,np.nan])
                    labels.append(np.argmax(y[ii[0]]))
                    pass
                else:
                    output = wsi_class(wsi_data_set)
                    outputs.append(output[0])
                    loss.append(output[1])
                    labels.append(np.argmax(y[ii[0]])) # OHE to binary

                del wsi_data_set
        
        test_out["Sample ID"] = unique_sample_ids # The samples are ordered since the loop goes through unique_sample_ids
        test_out["Output"] = [output[0] for output in outputs]
        test_out["Label"] = labels

        test_out.to_csv(f"D:\data\Outputs\WSI\PDAC_train_{TEST_SPLIT}_{SPLIT+1}.csv")
